In [3]:
import os.path
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import pandas as pd
import dlib
import numpy as np
import copy
import pickle

Loading mode, get test_img names etc.

In [6]:
# input labels.
label = pd.read_csv("/home/clydechx/FairFace/mydataset/celeba/validation_labels.csv")
imgs_path = "/home/clydechx/FairFace/mydataset/celeba/img_align_celeba"
img_names = []

# 遍历每个文件夹
for index, row in label.iterrows():
    # Get the file path
    file_path = row['image_id']

    # Use os.path.basename to get the file name without the directory
    # Add the file name to the img_names list
    img_names.append(file_path)
print(img_names)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

model_fair_7 = torchvision.models.resnet34(pretrained=True)
model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 2)
# if runs on a machine without GPU, should do map_location=torch.device('cpu')
#model_fair_7.load_state_dict(torch.load('fair_face_models/fairface_alldata_20191111.pt')) 
model_fair_7.load_state_dict(torch.load('/home/clydechx/FairFace/celebA_model.pt'))
model_fair_7 = model_fair_7.to(device)


# trans

trans = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

['162771.jpg', '162772.jpg', '162773.jpg', '162774.jpg', '162775.jpg', '162776.jpg', '162777.jpg', '162778.jpg', '162779.jpg', '162780.jpg', '162781.jpg', '162782.jpg', '162783.jpg', '162784.jpg', '162785.jpg', '162786.jpg', '162787.jpg', '162788.jpg', '162789.jpg', '162790.jpg', '162791.jpg', '162792.jpg', '162793.jpg', '162794.jpg', '162795.jpg', '162796.jpg', '162797.jpg', '162798.jpg', '162799.jpg', '162800.jpg', '162801.jpg', '162802.jpg', '162803.jpg', '162804.jpg', '162805.jpg', '162806.jpg', '162807.jpg', '162808.jpg', '162809.jpg', '162810.jpg', '162811.jpg', '162812.jpg', '162813.jpg', '162814.jpg', '162815.jpg', '162816.jpg', '162817.jpg', '162818.jpg', '162819.jpg', '162820.jpg', '162821.jpg', '162822.jpg', '162823.jpg', '162824.jpg', '162825.jpg', '162826.jpg', '162827.jpg', '162828.jpg', '162829.jpg', '162830.jpg', '162831.jpg', '162832.jpg', '162833.jpg', '162834.jpg', '162835.jpg', '162836.jpg', '162837.jpg', '162838.jpg', '162839.jpg', '162840.jpg', '162841.jpg', '1628

/home/clydechx/FairFace/env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/clydechx/FairFace/env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Start running the model.

In [9]:
# # Define a hook for all ReLU layers to compute the sum of activations of each filter
# activation = {}
# def relu_hook(module, input, output, layer_name):
#     activation[layer_name] = output.detach()
# # Register the hook for all ReLU layers
# for layer_name, layer in model.named_modules():
#     if isinstance(layer, torch.nn.ReLU):
#         layer.register_forward_hook(relu_hook, layer_name=layer_name)
num_filters = {}
activation = {}
filter_sums = {}
def get_activation(name):
    def hook(model, input, output):
        # Squeeze the tensor to remove the dimension with size 1
        squeezed_tensor = torch.squeeze(output.detach())
        activation[name] = squeezed_tensor
        # If squeezed_tensor has more than one dimension, flatten it
        if len(squeezed_tensor.shape) > 1:
            flattened_tensor = torch.flatten(squeezed_tensor, start_dim=1)
        else:
            flattened_tensor = squeezed_tensor
        # Sum along the second dimension if it exists, otherwise keep the tensor as is
        if flattened_tensor.dim() > 1:
            summed_tensor = flattened_tensor.sum(dim=1)
        else:
            summed_tensor = flattened_tensor
        # Convert the tensor to a list
        summed_list = summed_tensor.tolist()
        filter_sums[name] = summed_list
        num_filters[name] = len(summed_list)
    return hook

for name, layer in model_fair_7.named_modules():
    if isinstance(layer, torch.nn.ReLU):
        layer.register_forward_hook(get_activation(name))
    

In [11]:
# Define class labels for your model
classes = ['Male','Female']
# Define variables to track accuracy and class-wise accuracy
correct = 0
total = 0
class_correct = [0] * 2
class_total = [0] * 2

# Updated race and gender dictionaries

gender_dict = { 'Male': 0, 'Female': 1}

# New class map
my_class_map = {
    0:0,
    1:1
}

image_predictation_result = {cl: {} for cl in range(len(classes))}
activation_all_images = []
filtersum_all_images = []
miss_predicated_images = []
# Set the model to evaluation mode
model_fair_7.eval()
my_label = label
for index, img_name in enumerate(img_names):
    if index % 1000 == 0:
        print("Predicting... {}/{}".format(index, len(img_names)))
    img_name_in_label =  img_name
    label_row = my_label[my_label['image_id'] == img_name_in_label].iloc[0]
    
    # load image, run model.
    
    loading_path =  os.path.join(imgs_path,img_name)
    image = dlib.load_rgb_image(loading_path)
    image = trans(image)
    image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
    image = image.to(device)
    
    outputs = model_fair_7(image)
    outputs = outputs.cpu().detach().numpy()
    outputs = np.squeeze(outputs)
    
    
    gender_outputs = outputs[:2]
    
    
    gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))

    
    gender_pred = np.argmax(gender_score)

    predict_class = my_class_map[gender_pred]
    label_class_name =label_row['gender']
    label_class = classes.index(label_class_name)
    
    
    total +=1
    class_total[label_class] += 1
    image_predictation_result[label_class ][index] = predict_class
    filtersum_all_images.append(copy.deepcopy(filter_sums))
    
    # predict correctly 
    if predict_class == label_class:
        correct+=1
        class_correct[label_class]+=1
    # predict wrongly
    else:
        miss_predicated_images.append(index)
        print("Predicted label: {}, Ground truth: {}".format(predict_class, label_class))


Predicting... 0/19867
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 0
Predicting... 1000/19867
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 

Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicting... 2000/19867
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicting... 3000/19867
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 1, Ground truth: 0
Predicted label: 1, Ground truth: 0
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted label: 0, Ground truth: 1
Predicted labe

In [12]:
# Print the overall accuracy and accuracy for each class
print('size of test set: {}, accuracy of the network on the test set: {:.2f}%'.format(total, 100 * correct / total))
for i in range(2):
    print('Accuracy of {} : {:.0f}% of {} images'.format(classes[i], 100 * class_correct[i] / class_total[i], class_total[i]))
print("miss_predicated_images num = {}".format(len(miss_predicated_images)))

size of test set: 19867, accuracy of the network on the test set: 98.68%
Accuracy of Male : 99% of 8458 images
Accuracy of Female : 98% of 11409 images
miss_predicated_images num = 262


In [13]:
print(len(activation_all_images))
# print(len(activation_all_images[0]))
# print(len(activation_all_images[0]["features.22"][0]))
hooked_layer_names = list(activation.keys())
print(hooked_layer_names)
print(len(filtersum_all_images[0]))
print((filtersum_all_images[0]))

0
['relu', 'layer1.0.relu', 'layer1.1.relu', 'layer1.2.relu', 'layer2.0.relu', 'layer2.1.relu', 'layer2.2.relu', 'layer2.3.relu', 'layer3.0.relu', 'layer3.1.relu', 'layer3.2.relu', 'layer3.3.relu', 'layer3.4.relu', 'layer3.5.relu', 'layer4.0.relu', 'layer4.1.relu', 'layer4.2.relu']
17
{'relu': [8243.8203125, 2026.9300537109375, 5281.66748046875, 3328.39990234375, 2575.076171875, 2033.9508056640625, 2412.91015625, 13854.818359375, 9212.40625, 260.610107421875, 2887.80078125, 3379.99560546875, 1775.5634765625, 2220.5107421875, 3707.8916015625, 0.0, 0.0, 3208.03662109375, 1986.285888671875, 48.53357696533203, 3127.719482421875, 4273.6376953125, 3006.36474609375, 708.26416015625, 3706.4404296875, 3421.548583984375, 8510.8173828125, 9155.59765625, 4477.7255859375, 2777.8095703125, 4688.80322265625, 0.0, 3633.171142578125, 6959.6640625, 36.99211120605469, 0.0, 5038.3291015625, 2975.06640625, 8270.931640625, 2434.007080078125, 3781.805908203125, 0.0, 3450.06982421875, 212.94204711914062, 1228

In [14]:
def save_intermediate_data(data, file_path):
    """
    Save intermediate data to a file using pickle serialization.

    Args:
        data: The data to be saved.
        file_path (str): The path of the file to save the data.
    """
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)
# save intermediate results
save_intermediate_data(activation_all_images, "activation_CIFAR10_VGG16.pkl")
save_intermediate_data(image_predictation_result, "image_predictation_result.pkl")
save_intermediate_data(miss_predicated_images, "miss_predicated_images.pkl")
save_intermediate_data(hooked_layer_names, "hooked_layer_names.pkl")
save_intermediate_data(filtersum_all_images, "filtersum_all_images.pkl")
save_intermediate_data(num_filters, "num_filters.pkl")

In [15]:
def load_intermediate_data(file_path):
    """
    Load intermediate data from a file.

    Args:
        file_path (str): The path of the file to load the data from.

    Returns:
        The loaded data.
    """
    with open(file_path, 'rb') as file:
        data = pickle.load(file)
    return data


activation_all_images = load_intermediate_data("activation_CIFAR10_VGG16.pkl")
image_predictation_result = load_intermediate_data("image_predictation_result.pkl")
miss_predicated_images = load_intermediate_data("miss_predicated_images.pkl")
hooked_layer_names = load_intermediate_data("hooked_layer_names.pkl")
filtersum_all_images = load_intermediate_data("filtersum_all_images.pkl")
num_filters = load_intermediate_data("num_filters.pkl")

In [16]:
num_test_images = len(filtersum_all_images)
def find_outliers_zscore(data, threshold=1):
    """
    Find outliers in a list of values using z-score method.

    Args:
        data (list or numpy array): List of numeric values.
        threshold (float, optional): Z-score threshold for identifying outliers. Default is 3.

    Returns:
        list: List of outlier values.
    """
    # Convert data to numpy array
    data = np.array(data)
    if np.std(data) == 0:
        return []
    # Calculate z-scores
    z_scores = (data - np.mean(data)) / np.std(data)
    # Find outliers based on the threshold
    # outliers = data[np.abs(z_scores) > threshold]
    outliers = np.where(z_scores < -threshold)[0]
    return outliers

In [17]:
# find outlier for all mis_predicated images:
# cat (3)
class_id = 1
all_image_class = image_predictation_result[class_id].keys()
wrong_images = [x for x in all_image_class if image_predictation_result[class_id][x] != class_id]
print("num of wrong images in class {} is {}".format(class_id, len(wrong_images)))

num of wrong images in class 1 is 178


In [18]:
z_score_threshold = 1.0

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    for image in wrong_images:
        # print("image # {}".format(image))
        outliers = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        if len(outliers) > 0:
            print("image # {}, layer # {} has outliers {}".format(image, layer_name, outliers))



layer relu
image # 37, layer # relu has outliers [12 15 16 23 31 34 35 41 45 46 47 48 49 50 57 60 63]
image # 290, layer # relu has outliers [15 16 23 31 34 35 41 46 47 48 49 50 57 60 63]
image # 398, layer # relu has outliers [15 16 19 31 34 35 41 45 46 47 48 49 57 60 63]
image # 428, layer # relu has outliers [ 9 15 16 19 31 34 35 41 45 46 47 48 49 57 60 63]
image # 600, layer # relu has outliers [ 9 15 16 19 23 31 34 35 41 45 46 47 48 49 57 60 63]
image # 660, layer # relu has outliers [ 9 15 16 19 23 31 34 35 41 45 46 47 48 49 50 57 60 63]
image # 759, layer # relu has outliers [15 16 31 34 35 41 46 47 48 49 57 60 63]
image # 871, layer # relu has outliers [ 9 15 16 19 23 31 34 35 41 45 46 47 48 49 57 60 63]
image # 1037, layer # relu has outliers [15 16 23 31 34 35 41 46 47 48 49 57 60 63]
image # 1085, layer # relu has outliers [15 16 31 34 35 41 45 46 47 49 57 60 63]
image # 1200, layer # relu has outliers [12 15 16 19 23 31 34 35 41 45 46 47 48 49 57 60 63]
image # 1236, layer 

## activation level for the whole test set


In [19]:
print(num_filters)

{'relu': 64, 'layer1.0.relu': 64, 'layer1.1.relu': 64, 'layer1.2.relu': 64, 'layer2.0.relu': 128, 'layer2.1.relu': 128, 'layer2.2.relu': 128, 'layer2.3.relu': 128, 'layer3.0.relu': 256, 'layer3.1.relu': 256, 'layer3.2.relu': 256, 'layer3.3.relu': 256, 'layer3.4.relu': 256, 'layer3.5.relu': 256, 'layer4.0.relu': 512, 'layer4.1.relu': 512, 'layer4.2.relu': 512}


In [20]:
# outliers for each layer for each image
z_score_threshold = 2
all_outliers_for_image = {x: {} for x in hooked_layer_names}
num_times_being_outlier = {}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    num_times_being_outlier[layer_name] = [0] * num_filters[layer_name]
    for image in range(num_test_images):
        all_outliers_for_image[layer_name][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        for k in all_outliers_for_image[layer_name][image]:
            num_times_being_outlier[layer_name][k] += 1

layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [21]:
non_activation_ratio_whole_data = {}
ratio_threshold = 0.01
for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    s = {x: num_times_being_outlier[layer_name][x] / num_test_images for x in range(num_filters[layer_name]) if num_times_being_outlier[layer_name][x] > 0}
    non_activation_ratio_whole_data[layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}
    # non_activation_ratio_whole_data[layer_name] = [x/num_test_images for x in num_times_being_outlier[layer_name]]



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [22]:
for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_whole_data[layer_name])



relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {}
layer2.0.relu {}
layer2.1.relu {19: 0.010016610459556048, 50: 0.0101676146373383}
layer2.2.relu {60: 0.016107112296773543}
layer2.3.relu {60: 0.029093471586047213, 105: 0.04817033271253838}
layer3.0.relu {43: 0.070317612120602, 121: 0.03568732068253888, 144: 0.02612372275632959}
layer3.1.relu {118: 0.015452760859717118, 144: 0.18533246086475058, 146: 0.1447123370413248, 151: 0.11889062264055972, 161: 0.0679518800020134, 180: 0.06598882569084412, 197: 0.044244224090199825, 251: 0.03875773896411134}
layer3.2.relu {91: 0.016409120652338047, 118: 0.01847284441536216, 151: 0.037197362460361406, 161: 0.14274928273015552, 225: 0.03473096088991796}
layer3.3.relu {161: 0.030402174460160063}
layer3.4.relu {}
layer3.5.relu {}
layer4.0.relu {122: 0.010771631348467307, 300: 0.014546735793023607}
layer4.1.relu {}
layer4.2.relu {}


## non-activation times ratio for different classes, for right/wrong

In [23]:
# outliers for each layer for each image
z_score_threshold = 2
all_outliers_for_miss_predicated_image = {x: {} for x in hooked_layer_names}
num_times_being_outlier_miss_predicated_images = {}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    num_times_being_outlier_miss_predicated_images[layer_name] = [0] * num_filters[layer_name]
    for image in wrong_images:
        all_outliers_for_miss_predicated_image[layer_name][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
        for k in all_outliers_for_miss_predicated_image[layer_name][image]:
            num_times_being_outlier_miss_predicated_images[layer_name][k] += 1

layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [24]:
non_activation_ratio_miss_predicated_image = {}
num_miss_predicated_images = len(wrong_images)

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    s = {x: num_times_being_outlier_miss_predicated_images[layer_name][x] / num_miss_predicated_images for x in range(num_filters[layer_name]) if num_times_being_outlier_miss_predicated_images[layer_name][x] > 0}
    non_activation_ratio_miss_predicated_image[layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}


for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_miss_predicated_image[layer_name])



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {}
layer2.0.relu {}
layer2.1.relu {}
layer2.2.relu {60: 0.011235955056179775}
layer2.3.relu {60: 0.02247191011235955, 105: 0.056179775280898875}
layer3.0.relu {43: 0.02247191011235955, 144: 0.011235955056179775, 241: 0.02247191011235955}
layer3.1.relu {91: 0.016853932584269662, 118: 0.011235955056179775, 144: 0.14606741573033707, 146: 0.15730337078651685, 151: 0.05056179775280899, 161: 0.016853932584269662, 180: 0.06179775280898876, 197: 0.028089887640449437, 251: 0.05056179775280899}
layer3.2.relu {91: 0.011235955056179775, 103: 0.016853932584269662, 118: 0.011235955056179775, 151: 0.056179775280898875, 1

## non-activation times for different classes

In [25]:
all_outliers_for_classes = {x: {y: {} for y in range(len(classes))} for x in hooked_layer_names}
num_times_being_outlier_classes = {y: {} for y in range(len(classes))}

for layer_name in hooked_layer_names:
    print("layer {}".format(layer_name))
    for cla in range(len(classes)):
        num_times_being_outlier_classes[cla][layer_name] = [0] * num_filters[layer_name]
        for image in image_predictation_result[cla]:
            all_outliers_for_classes[layer_name][cla][image] = find_outliers_zscore(filtersum_all_images[image][layer_name], threshold=z_score_threshold)
            for k in all_outliers_for_classes[layer_name][cla][image]:
                num_times_being_outlier_classes[cla][layer_name][k] += 1



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [26]:
non_activation_ratio_classes = {y: {} for y in range(len(classes))}
ratio_threshold = 0.01
for cla in range(len(classes)):
    for layer_name in hooked_layer_names:
        print("layer {}".format(layer_name))
        s = {x: num_times_being_outlier_classes[cla][layer_name][x] / len(image_predictation_result[cla]) for x in range(num_filters[layer_name]) if num_times_being_outlier_classes[cla][layer_name][x] > 0}
        non_activation_ratio_classes[cla][layer_name] = {k: v for k, v in s.items() if v > ratio_threshold}



layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu
layer relu
layer layer1.0.relu
layer layer1.1.relu
layer layer1.2.relu
layer layer2.0.relu
layer layer2.1.relu
layer layer2.2.relu
layer layer2.3.relu
layer layer3.0.relu
layer layer3.1.relu
layer layer3.2.relu
layer layer3.3.relu
layer layer3.4.relu
layer layer3.5.relu
layer layer4.0.relu
layer layer4.1.relu
layer layer4.2.relu


In [28]:
# for cla in range(len(classes)):
#     for layer_name in hooked_layer_names:
#         print(layer_name, non_activation_ratio_classes[cla][layer_name])

# less non-activation classes: 0, 2, 4, 8
# many: 1, 3, 5, 6, 7, 9
cla = 0
print('Accuracy of {} : {:.0f}% of {} images'.format(classes[cla], 100 * class_correct[cla] / class_total[cla], class_total[cla]))
for layer_name in hooked_layer_names:
    print(layer_name, non_activation_ratio_classes[cla][layer_name])


Accuracy of Male : 99% of 8458 images
relu {}
layer1.0.relu {}
layer1.1.relu {}
layer1.2.relu {}
layer2.0.relu {}
layer2.1.relu {50: 0.011586663513833058}
layer2.2.relu {}
layer2.3.relu {105: 0.052731142113974935}
layer3.0.relu {43: 0.0854812012296051, 121: 0.04646488531567747, 144: 0.02080870182076141, 251: 0.01856230787420194}
layer3.1.relu {118: 0.019389926696618588, 144: 0.18077559706786475, 146: 0.21565381886970916, 151: 0.06148025537952235, 161: 0.06680066209505793, 180: 0.11468432253487822, 197: 0.03204067155355876, 251: 0.07566800662095058}
layer3.2.relu {91: 0.013478363679356823, 103: 0.010640813431071176, 118: 0.023173327027666116, 151: 0.03133128399148735, 161: 0.14471506266256798, 182: 0.011231969732797352, 225: 0.02199101442421376}
layer3.3.relu {161: 0.034168834239772995, 182: 0.01075904469141641}
layer3.4.relu {}
layer3.5.relu {}
layer4.0.relu {}
layer4.1.relu {}
layer4.2.relu {}
